# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues

apk_dir = "../../../Downloads/"

# APK Files to load
file_paths = (
    "com.snapchat.android_11.3.0.58-2113_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.snapchat.android_11.3.5.66-2114_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


file_paths = (
    "com.instagram.android_163.0.0.45.122-250742107_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.instagram.android_164.0.0.46.123-252055936_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk"
)
file_paths = tuple(map(lambda apk_name: apk_dir + apk_name, file_paths))

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_163.0.0.45.122-250742107_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_164.0.0.46.123-252055936_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Fields

Defining Elements that GraphGuard should try to match (obviously requires full class names).

GraphGuard has full Akrolyb support. Read about it at [Akrolyb Interoptability](#Interoptability-with-Akrolyb). Using this is optional. In case you don't want to use this functionality, do not run the few last cells that are related to Akrolyb Support and Automation


* For using Akrolyb Support:
    * Use CodeGen in Akrolyb to generate a Python-compatible tuple of declarations that you can copy paste into `named_c_decs`, `named_f_decs` and `named_f_decs`.
    * Specify the correct file for auto replacing values, the "declarations files".
    * Types of the variables:
        * `named_c_decs` in form of a `Dict[str, str]`
        * `named_m_decs` in form of a `Dict[str, MethodDec]`
        * `named_f_decs` in form of a `Dict[str, FieldDec]`


* For not using Akrolyb, and just simply try and match the values (without Auto-Replacing):
    * Leave named_c_decs, named_m_decs, named_f_decs empty.
    * Assign the declarations directly to c_decs, m_decs, f_decs further below by uncommenting the existing cell.
    * Types of the variables:
        * `c_decs` in form of a `Tuple[str]`
        * `m_decs` in form of a `Tuple[MethodDec]`
        * `f_decs` in form of a `Tuple[FieldDec]`

In [5]:
# Classes of the Methods and Fields are automatically merged with c_decs

named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPITEM': 'X.99q',
    'COMMENT_LONGCLICK_HANDLER': 'X.3Dq',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.38V',
    'DM_HOLDER': 'X.4t0',
    'DM_REPLY': 'X.3oE',
    'DM_VOICEMESSAGE': 'X.4na',
    'DMVIDEOPLAYER': 'X.9Bf',
    'FEEDITEM': 'X.1XB',
    'FRAGMENT': 'androidx.fragment.app.Fragment',
    'HEADERGENERATOR': 'X.0SW',
    'HTTPREQUESTTASKFACTORY': 'X.0pw',
    'INAPP_NOTIFICATION': 'X.6du',
    'IGTV_VIEWER_FRAGMENT': 'X.7DB',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.132',
    'JACKSON_CREATOR': 'X.0jN',
    'LIVE_POST': 'X.2Cx',
    'MAINFEED': 'X.1Rx',
    'MEDIAITEM': 'X.1XG',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.265',
    'NOTIFICATION_HANDLER': 'X.5Es',
    'PHOTOTIMERCONTROLLER': 'X.2qR',
    'POSTDETAIL': 'X.1fW',
    'POSTGESTURE_SIMPLEPOST': 'X.1wh',
    'POSTGESTURE_CAROUSELIMAGE': 'X.2CD',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2RK',
    'POSTVIEW': 'X.1fc',
    'PROFILE': 'X.0lo',
    'PROFILEDETAIL': 'X.30Q',
    'PROFILEPIC_LONGCLICK': 'X.30Z',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.2JL',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.0nv',
    'NOTIFICATION_RUNNABLE': 'X.6dv',
    'SERIALSCHEDULER': 'X.0jE',
    'SESSION': 'X.0Nb',
    'SIMPLE_EXECUTOR': 'X.3wg',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1Xv',
    'VIDEOSETTINGSMANAGER': 'X.1in',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [6]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPITEM_CONSTRUCTOR': MethodDec('X.99q', '<init>', 'X.28i', 'X.9Ak', 'X.0Nb', 'com.instagram.clips.intf.ClipsViewerConfig', 'android.view.View', 'X.94R', 'X.99u'),
    'COMMENT_ONLONGCLICK': MethodDec('X.3Dq', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1Hx', 'androidx.fragment.app.FragmentActivity', 'X.0Nb', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Nb'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Nb'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0E', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A05', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ADVANCETONEXT': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0H', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'boolean'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.38V', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.4t0', 'A00'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.3oE', 'A0L', 'X.3oE', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.3oE', 'A0H', 'X.3oE'),
    'DMVIDEOPLAYER_SEEKTO': MethodDec('X.9Bf', 'A02', 'int', 'boolean'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.1XB', 'A00', 'X.0jf'),
    'FRAGMENT_GETACTIVITY': MethodDec('androidx.fragment.app.Fragment', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0SW', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.0pw', 'A03', 'java.lang.String', 'X.0Nb'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.6du', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.7DB', 'A00', 'X.7DB'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.7DB', 'A0C', 'X.7DB'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.132', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0jN', 'A03', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0jN', '<init>', 'X.DS9'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.2Cx', 'A00', 'X.0kV', 'X.2Cy'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'ALj'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1Rx', 'configureActionBar', 'X.1Lw'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1XG', 'A01', 'X.0jf', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.0kV', 'X.1XG'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.265', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.5Es', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.2qR', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1fW', 'A06', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.1wh', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.2CD', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2RK', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1fc', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.30Q', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0lo', 'A04', 'X.0Nb', 'X.0kD'),
    'PROFILE_ONLONGCLICK': MethodDec('X.30Z', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.2JL', 'A0U', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'BPj', 'java.lang.Object'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0j', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0O', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0t', 'X.1qZ', 'X.1vD'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.0nv', 'A00', 'X.0nu', 'X.0tX'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.6dv', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.0jE', 'A02', 'X.0nj'),
    'SESSION_GETTOKEN': MethodDec('X.0Nb', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.3wg', 'onFail', 'X.2Cw'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.3wg', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.3wg', '<init>', 'X.0Nb'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1Xv', 'A03', 'X.0kV', 'X.0kC'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1in', 'A00', 'X.0Nb'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [7]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1Lv', 'A09'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.28i', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.3Dq', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1Yw', 'A0a'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_VIDEOPLAYER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mVideoPlayer'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.38V', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.4tk', 'A07'),
    'DMVIDEOPLAYER_CONTROLLER': FieldDec('X.AL2', 'A04'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.4na', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.1XB', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0qa', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.7DB', 'A0Y'),
    'MAINFEED_SESSION': FieldDec('X.1Rx', 'A0Y'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1Wt', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.1vD', 'A0A'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.1vD', 'A09'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.0vX', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.1ex', 'A01'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.6du', 'A00'), FieldDec('X.6dv', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.6du', 'A01'), FieldDec('X.6dv', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1uR', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1uU', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.30V', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.30V', 'A05'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.30V', 'A06'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.30V', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.30X', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.0vU', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.2nZ', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1E'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.1ev', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.1vD', 'A0E')
}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

In [9]:
# Without using Akrolyb's Auto-Replacing Values:

# c_decs = tuple()
# m_decs = tuple()
# f_decs = tuple()

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [10]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [11]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [12]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [13]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

* Found multiple candidates for Method X.99q#<init>(X.28i, X.9Ak, X.0Nb, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.94R, X.99u)
+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A05(com.instagram.direct.visual.DirectVisualMessageViewerController) -> com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController)
+ Found single candidate for Method. Considering it a match 
	X.265#A01() -> X.1nR#A01()
+ Found single candidate for Method. Considering it a match 
	X.0SW#A00(android.content.Context) -> X.0SW#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0t(X.1qZ, X.1vD) -> com.instagram.reels.fragment.ReelViewerFragment#A0p(X.1pp, X.1vm)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActiv

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [14]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

.. Considering ambiguous match by UniqueString as match with a majority of 0.97 (LX/1Rx; -> LX/1RC;)
.. Considering ambiguous match by UniqueString as match with a majority of 0.97 (LX/7DB; -> LX/9Uw;)
+ Found single candidate for Class. Considering it a match! 
	LX/0Nb; -> LX/0Ne;
+ Found single candidate for Class. Considering it a match! 
	LX/0jE; -> LX/0iw;
+ Found single candidate for Class. Considering it a match! 
	LX/0pw; -> LX/0ru;
+ Found single candidate for Class. Considering it a match! 
	LX/0qa; -> LX/0zQ;
+ Found single candidate for Class. Considering it a match! 
	LX/0vU; -> LX/1Kq;
+ Found single candidate for Class. Considering it a match! 
	LX/0vX; -> LX/1Ks;
+ Found single candidate for Class. Considering it a match! 
	LX/132; -> LX/129;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer;
+ Found single candidate for Class. Considerin

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [15]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/3Dq; -> LX/3Sh;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/9Bf; -> LX/5Uy;
+ Found single candidate for Class. Considering it a match! 
	LX/0jN; -> LX/0j5;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/2qR; -> LX/2yZ;
+ Found single candidate for Class. Considering it a match! 
	LX/1wh; -> LX/239;
* Found multiple candidates for Class LX/2CD;
* Found multiple candidates for Class LX/2RK;
+ Found single candidate for Class. Considering it a match! 
	LX/1fc; -> LX/1ei;
+ Found single candidate for Class. Considering it a match! 
	LX/0lo; -> LX/0l

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [16]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.99q#<init>(X.28i, X.9Ak, X.0Nb, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.94R, X.99u)
Class not resolved for Method X.38V#onLongPress(android.view.MotionEvent)
Class not resolved for Method X.4t0#A00()
Class not resolved for Method X.2CD#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2RK#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	X.3Dq#onLongPress(android.view.MotionEvent) -> X.3Sh#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1Hx, androidx.fragment.app.FragmentActivity, X.0Nb, android.os.Bundle) -> com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1Gp, androidx.fragment.app.FragmentActivity, X.0Ne, andr

	X.1fW#A06(X.1uR, X.1XG, X.1vU, int, boolean, java.lang.String, X.0Nb, X.1RV, java.lang.Integer, X.1jK, X.1SB) -> X.1ed#A05(X.1tV, X.1f4, X.20N, int, boolean, java.lang.String, X.0Ne, X.1Qk, java.lang.Integer, X.1i2, X.1RQ)
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0O(com.instagram.reels.fragment.ReelViewerFragment) -> com.instagram.reels.fragment.ReelViewerFragment#A0M(com.instagram.reels.fragment.ReelViewerFragment)
+ Found single candidate for Method. Considering it a match 
	X.0jE#A02(X.0nj) -> X.0iw#A02(X.0pz)
+ Found single candidate for Method. Considering it a match 
	X.0Nb#getToken() -> X.0Ne#getToken()
+ Found single candidate for Method. Considering it a match 
	X.1Xv#A03(X.0kV, X.0kC) -> X.1os#A03(X.0kE, X.0jz)
Could resolve 0 new Classes, 11 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [17]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0kC; -> LX/0jz;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1a4; -> LX/1vM;
+ Found single candidate for Class. Considering it a match! 
	LX/2Cy; -> LX/2DQ;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0qe; -> LX/14R;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/gesture/GestureManagerFrameLayout; -> Lcom/instagram/ui/gesture/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a 

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [18]:
print(len(accumulator.matching_cs), "/", len(c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

70 / 145 Classes were matched
52 / 60 Methods were matched
35 / 39 Fields were matched

Matching Classes:
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.265 -> X.1nR
• X.0SW -> X.0SW
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.3oE -> X.4Ij
• X.0nv -> X.0ns
• X.1in -> X.1hW
• X.2JL -> X.2MU
• X.3wg -> X.4QM
• X.6du -> X.BiP
• X.6dv -> X.BiQ
• X.0Nb -> X.0Ne
• X.0jE -> X.0iw
• X.0pw -> X.0ru
• X.0qa -> X.0zQ
• X.0vU -> X.1Kq
• X.0vX -> X.1Ks
• X.132 -> X.129
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1Lv -> X.1LR
• androidx.fragment.app.Fragment -> androidx.fragment.app.

• androidx.fragment.app.Fragment#getActivity() -> androidx.fragment.app.Fragment#getActivity()
• X.0pw#A03(java.lang.String, X.0Nb) -> X.0ru#A03(java.lang.String, X.0Ne)
• X.2Cx#A00(X.0kV, X.2Cy) -> X.2DP#A00(X.0kE, X.2DQ)
• X.1XG#A01(X.0jf, boolean) -> X.1f4#A01(X.0jN, boolean)
• com.instagram.feed.media.Media__JsonHelper#A00(X.0kV, X.1XG) -> com.instagram.feed.media.Media__JsonHelper#A00(X.0kE, X.1f4)
• X.5Es#A01(android.content.Context, java.lang.CharSequence, int) -> X.2dM#A01(android.content.Context, java.lang.CharSequence, int)
• X.1fW#A06(X.1uR, X.1XG, X.1vU, int, boolean, java.lang.String, X.0Nb, X.1RV, java.lang.Integer, X.1jK, X.1SB) -> X.1ed#A05(X.1tV, X.1f4, X.20N, int, boolean, java.lang.String, X.0Ne, X.1Qk, java.lang.Integer, X.1i2, X.1RQ)
• com.instagram.reels.fragment.ReelViewerFragment#A0O(com.instagram.reels.fragment.ReelViewerFragment) -> com.instagram.reels.fragment.ReelViewerFragment#A0M(com.instagram.reels.fragment.ReelViewerFragment)
• X.0jE#A02(X.0nj) -> X.0iw#

In [19]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(c_decs, r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    # No match for Method X.99q#<init>(X.28i, X.9Ak, X.0Nb, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.94R, X.99u),
    MethodDec('X.3Sh', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1Gp', 'androidx.fragment.app.FragmentActivity', 'X.0Ne', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Nb),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Nb),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direc

    # No Match Found for Class 'X.4t0'
    'X.4Ij'
    'X.53g'
    'X.5Uy'
    'X.1vG'
    'androidx.fragment.app.Fragment'
    'X.0SW'
    'X.0ru'
    'X.BiP'
    'X.9Uw'
    'com.instagram.realtimeclient.RealtimeClientManager'
    'X.129'
    'X.0j5'
    'X.2DP'
    'X.1RC'
    'X.1f4'
    'com.instagram.feed.media.Media__JsonHelper'
    'X.1nR'
    'X.2dM'
    'X.2yZ'
    'X.1ed'
    'X.239'
    # No Match Found for Class 'X.2CD'
    # No Match Found for Class 'X.2RK'
    'X.1ei'
    'X.0lY'
    'X.3B2'
    'X.3B9'
    'com.facebook.proxygen.JniHandler'
    'X.2MU'
    'com.instagram.reels.fragment.ReelViewerFragment'
    'X.0ns'
    'X.BiQ'
    'X.0iw'
    'X.0Ne'
    'X.4QM'
    'com.instagram.ui.swipenavigation.SwipeNavigationContainer'
    'X.1os'
    'X.1hW'
    'com.instagram.ui.widget.volume.VolumeIndicator',
)
f_decs = (
    FieldDec('X.1LR', 'A09'),
    FieldDec('X.2BL', 'A00'),
    FieldDec('X.3Sh', 'A04'),
    FieldDec('X.1pS', 'A0a'),
    FieldDec('com.instagram.direct.v

# Warnings

Shows some warnings that result in common bugs, so that the Hooks (or other kind of modifications applied) must be updated logic-wise. 

Currently supports: 
* Change in number of parameters 

In [20]:
for m_old, m_new in accumulator.matching_ms.items():
    def get_number_of_params(m):
        return len(tuple(get_pretty_params(str(m.get_descriptor()))))
    
    if get_number_of_params(m_old) != get_number_of_params(m_new):
        print("Warning: Number of parameters changed for", pretty_format_ma(m_old))

# Interoptability with Akrolyb

GraphGuard was build with Akrolyb in mind and has full Akrolyb support, meaning that it is not only capable of automatically matching the given declarations, but also automatically editing the "declaration file" to update matched declarations and mark unmatched items with a `/* TODO */` comment.

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. See [this](#List-of-Classes,-Methods-and-Fields) to learn how to use GraphGuard's Akrolyb Support.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```

In [21]:
file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/"
c_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"


file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/"
c_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [22]:
%autoreload
c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for X.99q#<init>(X.28i, X.9Ak, X.0Nb, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.94R, X.99u)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Nb)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Nb)
No matching Method found for X.38V#onLongPress(android.view.MotionEvent)
No matching Method found for X.4t0#A00()
No matching Method found for X.1Rx#configureActionBar(X.1Lw)
No matching Method found for X.2CD#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2RK#onDoubleTap(android.view.MotionEvent)
No matching Field found for X.38V#A01
No matching Field found for X.1uR#A06
No matching Field found for X.1uU#A02
No matching Field found for com.ins

In [23]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = /* TODO */ ClassDec("X.99q")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3Sh")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = /* TODO */ ClassDec("X.38V")

    override val DM_HOLDER = /* TODO */ ClassDec("X.4t0")

    override val DM_REPLY = ClassDec("X.4Ij")

    override val DM_VOICEMESSAGE = ClassDec("X.53g")

    override val DMVIDEOPLAYER = ClassDec("X.5Uy")

    override val FEEDITEM = ClassDec("X.1vG")

    override val FRAGMENT = 

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1LR"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A09")

    @FieldClass(["X.2BL"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.3Sh"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1pS"])
    override val COMMENT_TEXT = VariableDec<String>("A0a")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = VariableDec<Any>("A0D")

    @FieldClass(